In [442]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from skimage import io, morphology, img_as_bool, segmentation
from scipy import ndimage as ndi
from scipy.ndimage.morphology import binary_fill_holes

%matplotlib inline

In [528]:
def pad_resize(img):
    top = int((224 - img.shape[0])/2)
    left = int((224 - img.shape[1])/2)
    bottom = 224 - img.shape[0] - top
    right = 224 - img.shape[1] - left
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)
    img = img/255.
    img = cv2.resize(img, (50,50)) #KADD
    return img

def skeletonize(img):
    size = np.size(img)
    skel = np.zeros(img.shape,np.uint8)
    img = cv2.bitwise_not(img)
#     element = cv2.getStructuringElement(cv2.MORPH_CROSS,(1,1))
#     done = 0
#     while( done < 1 ):
#         eroded = cv2.erode(img,element)
#         temp = cv2.dilate(eroded,element)
#         temp = cv2.subtract(img,temp)
#         skel = cv2.bitwise_or(skel,temp)
#         img = eroded.copy()

#         zeros = size - cv2.countNonZero(img)
#         if zeros==size:#cv2.countNonZero(img) * 1 >= 0:#
#             done += 1
#     img = skel
    kernel = np.ones((3,3),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = 2)
    img = cv2.bitwise_not(erosion)
    return img

In [547]:
PATH = '../train_images_modified'
images = []
base_class = []
matra_class = []
total_class = []
for filename in os.listdir(PATH):
    if filename.endswith(".png"):
        img = cv2.imread(os.path.join(PATH,filename),0)
#         plt.imshow(img, cmap='gray')
#         plt.show()
        blur = cv2.GaussianBlur(img,(11,11),0)# KADD
        a,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)#KEDIT
#         plt.imshow(img, cmap='gray')
#         plt.show()
# #         kernel = np.ones((5,5),np.uint8)
#         img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#         plt.imshow(img, cmap='gray')
#         plt.show()
        #img = skeletonize(img)
        #plt.imshow(img, cmap='gray')
        #plt.show()
        img = pad_resize(img)
        blur = cv2.GaussianBlur(img,(9,9),0)# KADD
        char_arr = filename[:-4].split('_')[3:]
        if(len(char_arr)>0):
            images.append(img)
            char_arr = [int(i) for i in char_arr]
            total_class.append(char_arr)
            base = [i for i in char_arr if (i>=2308 and i<=2361) or (i==2384) or (i>=2392 and i<=2401) or (i>=2404 and i!=2416 and i!=2417)]
            if(len(base)>1):
                print(filename)
                print([i for i in base])
                print([hex(i) for i in base])
                print("=============")
            #base_class.append(base[0])
#             if(len(char_arr)>1):          
#                 matra_class.append(char_arr[1:])
#             if len(images)%100==0:
#                 print("{} Done".format(len(images)))
            
# print(len(images))
# print(len(char_class))
# print(images[0])
# print(char_class[0])

page0_7_5_2361_2404_2417.png
[2361, 2404]
['0x939', '0x964']
page1_16_9_2346_2429.png
[2346, 2429]
['0x92a', '0x97d']
page2_12_0_2352_2357.png
[2352, 2357]
['0x930', '0x935']
page2_7_0_2310_2350.png
[2310, 2350]
['0x906', '0x92e']
page2_7_10_2322_2342.png
[2322, 2342]
['0x912', '0x926']
page2_7_7_2310_2320_2357.png
[2310, 2320, 2357]
['0x906', '0x910', '0x935']
page2_8_0_2312_2342.png
[2312, 2342]
['0x908', '0x926']
page2_8_10_2322_2354.png
[2322, 2354]
['0x912', '0x932']
page2_8_3_2319_2325.png
[2319, 2325]
['0x90f', '0x915']
page2_8_4_2311_2361.png
[2311, 2361]
['0x907', '0x939']
page2_8_8_2310_2354.png
[2310, 2354]
['0x906', '0x932']
page2_9_0_2319_2342.png
[2319, 2342]
['0x90f', '0x926']
page2_9_17_2310_2332.png
[2310, 2332]
['0x906', '0x91c']
page2_9_2_2306_2318_2340.png
[2318, 2340]
['0x90e', '0x924']
page2_9_3_2310_2346.png
[2310, 2346]
['0x906', '0x92a']
page2_9_9_2322_2325.png
[2322, 2325]
['0x912', '0x915']
page3_0_0_2310_2423.png
[2310, 2423]
['0x906', '0x977']
page3_10_10_2

In [ ]:
plt.imshow(images[10],"gray")
plt.show()
print(len(images))

# KADD:

In [405]:
x_data = np.reshape(np.array(images), (-1, 50*50))
y_data = np.array(char_class)

df = pd.DataFrame(x_data, y_data)
df['LABEL'] = df.index

df_no_label = df.drop(columns='LABEL')

In [397]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split

#print(df.head())
#print(df_no_label)
#print(df['LABEL'])

X_train,X_val,y_train,y_val = train_test_split(df_no_label, df['LABEL'])

print(X_train.shape)
print(X_val.shape)

clf = RandomForestClassifier(max_features='auto', n_estimators=10, max_depth=20)
clf.fit(X_train, y_train)

scores = cross_val_score(clf, df_no_label, df['LABEL'], cv=3)
print(scores)
scores_f1 = f1_score(clf.predict(X_train),y_train,average='weighted')
print(scores_f1)
scores_f1 = f1_score(clf.predict(X_val),y_val,average='weighted')
print(scores_f1)

(1423, 2500)
(475, 2500)


/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[ 0.15372907  0.18600954  0.17320261]
0.964820312972
0.20194229471


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [192]:
len(total_class)

1898

In [193]:
import itertools
char_class_types = list(itertools.chain.from_iterable(char_class))
#char_class_types = set([int(i) for i in char_class_types])
char_class_types = set(char_class_types)
len(char_class_types)

57

In [194]:
matra_class_types = list(itertools.chain.from_iterable(matra_class))
#matra_class_types = set([int(i) for i in matra_class_types])
matra_class_types = set(matra_class_types)
len(matra_class_types)

47

In [195]:
total_char_set = char_class_types.union(matra_class_types)
len(total_char_set)

77

In [196]:
total_char_list = sorted(list(total_char_set))

In [197]:
total_char_dict = {total_char_list[i]:i for i in range(len(total_char_list))}

In [198]:
list(itertools.chain.from_iterable(char_class))

['2313',
 '2352',
 '2340',
 '2312',
 '2344',
 '2361',
 '2312',
 '2344',
 '2346',
 '2325',
 '2340',
 '2354',
 '2347',
 '2360',
 '2352',
 '2325',
 '2348',
 '2315',
 '2346',
 '2361',
 '2350',
 '2332',
 '2354',
 '2354',
 '2346',
 '2351',
 '2344',
 '2348',
 '2306',
 '2361',
 '2340',
 '2361',
 '2352',
 '2350',
 '2352',
 '2411',
 '2327',
 '2344',
 '2315',
 '2357',
 '2315',
 '2346',
 '2361',
 '2350',
 '2346',
 '2340',
 '2344',
 '2350',
 '2351',
 '2327',
 '2359',
 '2361',
 '2352',
 '2360',
 '2348',
 '2344',
 '2346',
 '2361',
 '2350',
 '2360',
 '2332',
 '2340',
 '2360',
 '2351',
 '2344',
 '2325',
 '2360',
 '2352',
 '2336',
 '2354',
 '2352',
 '2352',
 '2404',
 '2346',
 '2312',
 '2354',
 '2360',
 '2361',
 '2343',
 '2361',
 '2354',
 '2312',
 '2311',
 '2344',
 '2325',
 '2352',
 '2340',
 '2346',
 '2361',
 '2360',
 '2327',
 '2334',
 '2340',
 '2346',
 '2344',
 '2360',
 '2346',
 '2352',
 '2360',
 '2360',
 '2352',
 '2361',
 '2315',
 '2346',
 '2325',
 '2407',
 '2342',
 '2352',
 '2325',
 '2340',
 '2332',
 

In [79]:
import keras
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

le = LabelEncoder()
le.fit(list(itertools.chain.from_iterable(char_class)))
y_labeled = le.transform(list(itertools.chain.from_iterable(char_class)))

In [80]:
y_train2 = []
for char in total_class:
    y_train2_char = np.zeros(len(total_char_list))
    for element in char:
        y_train2_char[total_char_dict[element]] = 1
    y_train2.append(y_train2_char)

In [81]:
y_train2[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.])

In [82]:
y_train2 = np.array(y_train2)
np.array(images).shape

(1963, 224, 224)

In [83]:
X_train = np.reshape(np.array(images),(-1,224,224,1))
#char_int = [int(i) for i in list(itertools.chain.from_iterable(char_class))]
y_train = np_utils.to_categorical(y_labeled)

In [84]:
len(y_train2[0])

80

In [96]:
model = Sequential()

model.add(Dense(256,input_shape=(224*224,),activation = 'sigmoid'))
model.add(Dense(128,activation = 'sigmoid'))
model.add(Dense(128,activation = 'sigmoid'))
model.add(Dense(len(total_char_set),activation = 'sigmoid'))

print(model.summary())
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_score(theta=0.5),'accuracy'])
X_train = X_train.reshape((-1,224*224))
model.fit(X_train,y_train2,epochs=10,batch_size=32,validation_split=0.3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 256)               12845312  
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_24 (Dense)             (None, 80)                10320     
Total params: 12,905,040
Trainable params: 12,905,040
Non-trainable params: 0
_________________________________________________________________
None
Train on 1374 samples, validate on 589 samples
Epoch 1/10
1374/1374 [==============================] - 14s 10ms/step - loss: 6.4939 - score: 0.0886 - acc: 0.0182 - val_loss: 6.6137 - val_score: 0.0999 - val_acc: 0.0051
Epoch 2/10
1374/1374 [================

KeyboardInterrupt: 

In [90]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten

model = Sequential()

model.add(Conv2D(16,(3,3),input_shape = (224,224,1),activation = 'relu'))
model.add(Conv2D(16,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256,activation = 'sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(len(total_char_set),activation = 'sigmoid'))

In [91]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 222, 222, 16)      160       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 220, 220, 16)      2320      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 108, 108, 32)      4640      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 106, 106, 32)      9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 53, 53, 32)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 26, 26, 32)        0         
__________

In [87]:
import keras.backend as K

def f1_score(theta):
    def score(y_true, y_pred):

        y_thresh = K.cast(K.greater(y_pred,theta),K.floatx())

        true_pos =  K.sum(y_true * y_thresh)
        false_pos = K.sum(y_true * (1. - y_thresh))
        false_neg = K.sum((1. - y_true) * y_thresh)

        precision = true_pos / (true_pos + false_pos)
        recall = true_pos / (true_pos + false_neg)
        
        f1_score_val = 2 * (precision * recall) / (precision + recall)
        return f1_score_val
    return score

def custom_metric(y_true, y_pred):
    return K.cast(K.equal(y_true,
                          K.round(y_pred)),
                  K.floatx())


In [88]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_score(theta=0.5)])

In [89]:
model.fit(X_train,y_train2,epochs=10,batch_size=32,validation_split=0.3)

Train on 1374 samples, validate on 589 samples
Epoch 1/10
1374/1374 [==============================] - 138s 101ms/step - loss: 6.3004 - score: 0.0907 - val_loss: 6.6307 - val_score: 0.1009
Epoch 2/10
1374/1374 [==============================] - 139s 101ms/step - loss: 6.0385 - score: 0.1098 - val_loss: 6.6040 - val_score: 0.1103
Epoch 3/10
1374/1374 [==============================] - 135s 98ms/step - loss: 6.0297 - score: 0.1186 - val_loss: 6.6003 - val_score: 0.1219
Epoch 4/10
1374/1374 [==============================] - 144s 105ms/step - loss: 6.0206 - score: 0.1207 - val_loss: 6.5908 - val_score: 0.1273
Epoch 5/10
  96/1374 [=>............................] - ETA: 1:51 - loss: 5.9091 - score: 0.1123

KeyboardInterrupt: 

In [ ]:
preds = model.predict(X_train)

In [157]:
labels = np.zeros(preds[0].shape)
labels[preds[5]>0.5] = 1
labels[0]

0.0